In [ ]:
from pathlib import Path
from tqdm import tqdm
import math

import numpy as np
import pandas as pd

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
from torchvision import transforms

from leaf.dta import LeafDataset, GetPatches, TransformPatches, RandomGreen, LeafDataLoader
from leaf.model import LeafModel, train_one_epoch, validate_one_epoch, warmup
from torch.utils.data import RandomSampler, DataLoader

from torch.optim.lr_scheduler import CyclicLR

from torch.utils.tensorboard import SummaryWriter


# Transforms with normalizations for imagenet
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor(),
        RandomGreen(224, 224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'warmup': transforms.Compose([
        transforms.ToTensor(),
        RandomGreen(224, 224),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'patches': transforms.Compose([
        transforms.ToTensor(),
        transforms.RandomHorizontalFlip(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    ])
}

In [ ]:
output_dir = Path("/mnt/hdd/leaf-disease-outputs")
output_dir.mkdir(exist_ok=True)
logging_dir = Path("/mnt/hdd/leaf-disease-runs")
logging_dir.mkdir(exist_ok=True)

batch_size = 16
val_batch_size = 32

num_workers = 4

patches_logging_steps = 1000
train_logging_steps = 250
save_checkpoints = False

train_dset = LeafDataset("./data/train_images", "./data/train_images/labels.csv", transform=data_transforms["train"])
patches_dset = LeafDataset("./data/patches_train", "./data/patches_train/labels.csv", extended_labels=True, transform=data_transforms["patches"])
warmup_dset = LeafDataset("./data/train_images", "./data/train_images/labels.csv", transform=data_transforms["warmup"])
val_dset = LeafDataset("./data/patches_val", "./data/patches_val/labels.csv", extended_labels=True, transform=data_transforms["val"])

train_dataloader = DataLoader(train_dset, batch_size=batch_size, sampler=RandomSampler(train_dset, replacement=True), num_workers=num_workers, pin_memory=True)
patches_dataloader = LeafDataLoader(patches_dset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
val_dataloader = LeafDataLoader(val_dset, batch_size=val_batch_size, shuffle=False, num_workers=num_workers)

leaf_model = LeafModel("tf_efficientnet_b4_ns", model_prefix="MONEVE", output_dir=output_dir, logging_dir=logging_dir)

# optimizer = Adam(leaf_model.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
optimizer = Adam(leaf_model.model.parameters(), lr=1e-6, weight_decay=1e-6)
scheduler = None
#scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=T_0, T_mult=1, eta_min=min_learning_rate, last_epoch=-1)
leaf_model.update_optimizer_scheduler(optimizer, scheduler)

In [ ]:
leaf_model.load_checkpoint("MONEVE_epoch_2")

In [ ]:
leaf_model.model_prefix = "THUR"

In [ ]:
learning_rate = 1e-4
#min_learning_rate = 1e-6
weight_decay = 5e-7

#T_0 = 10

In [ ]:
optimizer = Adam(leaf_model.model.parameters(), lr=learning_rate, weight_decay=weight_decay)
scheduler = None

In [ ]:
# optimizer = SGD(leaf_model.model.parameters(), lr=1e-3, weight_decay=weight_decay)
# scheduler = CyclicLR(optimizer, 1e-6, 1e-3, step_size_up=2*len(train_dataloader), mode="exp_range", gamma=0.9)

In [ ]:
leaf_model.update_optimizer_scheduler(optimizer, scheduler)

In [ ]:
run_name = "THUR-10-epochs"
for epoch in range(10):
    epoch_name = f"{run_name}-{epoch}"
    train_one_epoch(leaf_model, train_dataloader, log_steps=200, save_at_log_steps=save_checkpoints, epoch_name=epoch_name)
    val_loss, raw_val_acc, val_acc = validate_one_epoch(leaf_model, val_dataloader, ensemble_patches=True)
    print(f"Validation after epoch {epoch}: loss {val_loss}, raw acc {raw_val_acc}, acc {val_acc}")
    tb_writer = SummaryWriter(log_dir=leaf_model.logging_model_dir / epoch_name)
    val_step = epoch * len(train_dataloader)
    tb_writer.add_scalar("loss/val", val_loss, val_step)
    tb_writer.add_scalar("raw_acc/val", raw_val_acc, val_step)
    tb_writer.add_scalar("acc/val", val_acc, val_step)
    tb_writer.close()
    leaf_model.save_checkpoint(f"{epoch_name}", epoch=epoch)